In [1]:
import pandas as pd
import numpy as np

import pulp


In [26]:
def dec2(number, decimals=2):
    return "{:.{}f}".format(number, decimals)

In [37]:
# race selection
year = 2024
circuit = 'abu-dhabi'

In [38]:
d_pts = pd.read_csv('data/' + str(year) + '_' + circuit + '_d_pts.csv')
c_pts = pd.read_csv('data/' + str(year) + '_' + circuit + '_c_pts.csv')

In [39]:
# add driver/constructor cost
d_cost_map = {'verstappen': 32.3,
             'norris': 27.5,
             'leclerc': 26.1,
             'piastri': 25.7,
             'sainz': 24.4,
             'hamilton': 25.8,
             'russell': 23.2,
             'perez': 23.2,
             'alonso': 15.3,
             'zhou': 9.6,
             'ocon': 14.4,
             'magnussen': 14.3,
             'gasly': 11.8,
             'stroll': 14.3,
             'hulkenberg': 11.8,
             'tsunoda': 11.9,
             'bottas': 7.1,
             'lawson': 10.9,
             'doohan':10.9,
             'colapinto': 7.4,
             'albon': 8.6,
             'ricciardo': 12,
             'bearman': 13.8,
             'sargeant': 4.6}

c_cost_map = {'mclaren': 27,
             'red-bull': 29.7,
             'ferrari': 25.3,
             'mercedes': 25.1,
             'aston-martin': 14.4,
             'haas': 12.2,
             'alpine': 12.1,
             'rb': 12.9,
             'kick-sauber': 7.5,
             'williams': 6.1}

d_pts['cost'] = d_pts['driverId'].map(d_cost_map)
c_pts = c_pts.rename(columns={'Unnamed: 0': 'constructorId'}) 
c_pts['cost'] = c_pts['constructorId'].map(c_cost_map)

In [54]:
def LP_Optimize_Team(drivers, constructors, budget=100, include_drivers=[], exclude_drivers=[],
                    include_constructors=[], exclude_constructors=[]):
    # set up linear programming optimization

    driver_idx = range(len(drivers))
    constructor_idx = range(len(constructors))
    ## Set variables
    D = pulp.LpVariable.dict('driver', driver_idx, cat=pulp.LpBinary)
    DRS = pulp.LpVariable.dict('drs', driver_idx, cat=pulp.LpBinary)
    C = pulp.LpVariable.dict('constructor', constructor_idx, cat=pulp.LpBinary)

    # set up objective function
    prob = pulp.LpProblem('FantasyFormula1', pulp.LpMaximize)
    prob += pulp.lpSum(drivers.loc[i, 'predicted_points'] * (D[i] + DRS[i]) for i in driver_idx) + \
       pulp.lpSum(constructors.loc[j, 'predicted_points'] * (C[j]) for j in constructor_idx)

    ## Set constraints
    # Exactly 5 drivers
    prob += sum(D[i] for i in driver_idx) ==  5
    
    # Exactly 2 constructors
    prob += sum(C[i] for i in constructor_idx) ==  2

    ## One DRS captain
    prob  += sum(DRS[i] for i in driver_idx) == 1

    for i in driver_idx:  # captain must also be on team
        prob += (D[i] - DRS[i]) >= 0

    # 100M Budget
    prob += sum(D[i] * drivers['cost'][i] for i in driver_idx) + \
        sum(C[j] * constructors['cost'][j] for j in constructor_idx) <= budget  # total cost

    # Optional: force driver and constructor picks
    for in_driver in include_drivers:
        prob  += sum(D[i] for i in driver_idx if i == in_driver) == 1
    
    for out_driver in exclude_drivers:
        prob  += sum(D[i] for i in driver_idx if i == out_driver) == 0
    
    solve_status = prob.solve() # Outputs 1 if successful
    
    # print optimal team
    opt_team = pd.DataFrame(columns=['Type','ID','DRS','Predicted_Points','cost'])
    for i in driver_idx:
        if pulp.value(D[i]) == 1:
            if pulp.value(DRS[i]) == 1:
                new_row = {'Type':'driver', 'ID':drivers.loc[i, 'driverId'], 'DRS': 1,
                           'Predicted_Points': drivers.loc[i, 'predicted_points']*2, 'cost': drivers.loc[i, 'cost']}
                opt_team.loc[len(opt_team)] = new_row
            else:
                new_row = {'Type':'driver', 'ID':drivers.loc[i, 'driverId'], 'DRS': 0,
                           'Predicted_Points': drivers.loc[i, 'predicted_points'], 'cost': drivers.loc[i, 'cost']}
                opt_team.loc[len(opt_team)] = new_row
    
    for i in constructor_idx:
        if pulp.value(C[i]) == 1:
            new_row = {'Type':'constructor', 'ID':constructors.loc[i, 'constructorId'], 'DRS': 0,
                           'Predicted_Points': constructors.loc[i, 'predicted_points'], 'cost': constructors.loc[i, 'cost']}
            opt_team.loc[len(opt_team)] = new_row
            
    if(solve_status == 1):
        print('Total Predicted Points: ' + str(round(pulp.value(prob.objective), 1))) # Show points total
        print('Total Cost: ' + str(round(opt_team['cost'].sum(), 1)))
    else:
        print('Solution not found')
        
    # round the predicted points column
    opt_team['Predicted_Points'] = round(opt_team['Predicted_Points'], 1)
            
    return opt_team

In [55]:


LP_Optimize_Team(d_pts, c_pts, budget=128, include_drivers=[], exclude_drivers=[],
                    include_constructors=[], exclude_constructors=[])

Total Predicted Points: 221.2
Total Cost: 127.4


,Type,ID,DRS,Predicted_Points,cost
0,driver,norris,1,57.0,27.5
1,driver,russell,0,25.0,23.2
2,driver,gasly,0,10.3,11.8
3,driver,bottas,0,2.9,7.1
4,driver,colapinto,0,2.4,7.4
5,constructor,mercedes,0,63.6,25.1
6,constructor,ferrari,0,59.9,25.3
